In [5]:
from bs4 import BeautifulSoup
import os
import csv

def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def read_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return lines

folder_path = 'raw_html'  # Specify the folder path containing the HTML files

output_folder = 'clean_text'  # Specify the folder where the clean text files will be saved
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

html_files = os.listdir(folder_path)  # Get the list of HTML files in the folder

for html_file in html_files:
    html_file_path = os.path.join(folder_path, html_file)
    lines = read_html_file(html_file_path)
    html_text = ''.join(lines)  # Join the lines into a single string
    clean_text = remove_html_tags(html_text)

    # Extract the filename from the html_file_path
    filename = os.path.splitext(html_file)[0]  # Extract the filename without extension
    output_file_path = os.path.join(output_folder, f'clean_text_{filename}.csv')

    with open(output_file_path, 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        for line in clean_text.splitlines():
            writer.writerow([line])

    print(f"Clean text saved in {output_file_path}.")


Clean text saved in clean_text\clean_text_Criminal Justice and Police Act 2001 (c. 16).csv.
Clean text saved in clean_text\clean_text_Human Rights Act 1998 (c. 42).csv.
Clean text saved in clean_text\clean_text_Public Order Act 1986 (c. 64).csv.
Clean text saved in clean_text\clean_text_Public Order Act 2023 (c. 15).csv.
Clean text saved in clean_text\clean_text_Serious Organised Crime and Police Act 2005 (c. 15).csv.
